<h2>Extract Transform Load<br>
and Training Notebook</h2><br>
Let's begin by importing the necessary libraries and modules.


In [1]:
import pandas as pd
import numpy as np
from numpy import savez_compressed
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import os
import joblib

Let's download the datasets from the cloud.

In [2]:
!gdown https://drive.google.com/uc?id=1oCIVZbIWmg2RvcCHcxko5alyZh19nGoF
!gdown https://drive.google.com/uc?id=1vCS6gDbsS3mVE5WLHM0nu-ww0GcI1ybN
!gdown https://drive.google.com/uc?id=10_pO2fGiAKSdNjO-p3d1dFo3VuY1juL5
!gdown https://drive.google.com/uc?id=1kSH1dteCOdFT5eFFM-1DCdbl7TZ4gXzS
!gdown https://drive.google.com/uc?id=1nRKTC6Pez1fRAMYkFIHQNLfB0Hhj5fok
!gdown https://drive.google.com/uc?id=1lA19-bvYJ_btc1WSXpKqU9L6nD3dpUjR
!gdown https://drive.google.com/uc?id=1Av_X9N5BB38edPLK0vtYIBQ9toUdDB1e
!gdown https://drive.google.com/uc?id=11u-MTRj8qiqC0rsB23F37lc4MjNh3Fhk

Downloading...
From: https://drive.google.com/uc?id=1oCIVZbIWmg2RvcCHcxko5alyZh19nGoF
To: /content/Monday-WorkingHours.pcap_ISCX.csv
177MB [00:03, 54.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vCS6gDbsS3mVE5WLHM0nu-ww0GcI1ybN
To: /content/Tuesday-WorkingHours.pcap_ISCX.csv
135MB [00:02, 51.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=10_pO2fGiAKSdNjO-p3d1dFo3VuY1juL5
To: /content/Wednesday-workingHours.pcap_ISCX.csv
225MB [00:03, 62.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1kSH1dteCOdFT5eFFM-1DCdbl7TZ4gXzS
To: /content/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
52.0MB [00:01, 41.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1nRKTC6Pez1fRAMYkFIHQNLfB0Hhj5fok
To: /content/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
83.1MB [00:01, 77.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lA19-bvYJ_btc1WSXpKqU9L6nD3dpUjR
To: /content/Friday-WorkingHours-Morning.pcap_ISCX.csv
58.3MB [00:00, 60.3MB

In [3]:
datasets = []

files = os.listdir()
for f in files:
    if f[-3:] == 'csv':
        datasets.append(f)
datasets

['Friday-WorkingHours-Morning.pcap_ISCX.csv',
 'Monday-WorkingHours.pcap_ISCX.csv',
 'Wednesday-workingHours.pcap_ISCX.csv',
 'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
 'Tuesday-WorkingHours.pcap_ISCX.csv',
 'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
 'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
 'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv']

We create the master dataframe.

In [4]:
df = pd.read_csv(datasets[0])

for i in range(1, len(datasets)):
  df_inter = pd.read_csv(datasets[i])
  df = pd.concat([df, df_inter], axis=0)
  
df = df.sample(frac = 1)

In [5]:
imp_features = ['Flow IAT Min',
 'Bwd IAT Mean',
 'Flow IAT Mean',
 'Init_Win_bytes_forward',
 'Subflow Fwd Bytes',
 'Total Length of Fwd Packets',
 'ACK Flag Count',
 'Fwd PSH Flags',
 'SYN Flag Count',
 'Fwd Packets/s',
 'Init_Win_bytes_backward',
 'Flow Duration',
 'Active Mean',
 'Fwd Packet Length Mean',
 'Bwd Packets/s',
 'PSH Flag Count',
 'Bwd Packet Length Std',
 'Average Packet Size',
 'Flow IAT Std',
 'Label']

In [6]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
263199,443,392236,18,30,887,39350,405,0,49.277778,111.13839,2896,0,1311.666667,582.541124,1.025836e+05,122.375305,8345.446809,21281.765880,112010,3,392236,23072.70588,31147.63471,112010,3,328074,11312.89655,34229.21447,143835,189,0,0,0,0,588,968,45.890739,76.484565,0,2896,821.163265,772.818121,597247.847800,0,0,0,1,0,0,0,0,1,838.270833,49.277778,1311.666667,588,0,0,0,0,0,0,18,887,30,39350,65535,122,6,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
406628,53,203,2,2,70,262,35,35,35.000000,0.00000,131,131,131.000000,0.000000,1.635468e+06,19704.433500,67.666667,105.153855,189,3,3,3.00000,0.00000,3,3,11,11.00000,0.00000,11,11,0,0,0,0,64,64,9852.216749,9852.216749,35,131,73.400000,52.581366,2764.800000,0,0,0,0,0,0,0,0,1,91.750000,35.000000,131.000000,64,0,0,0,0,0,0,2,70,2,262,-1,-1,1,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
133924,53,204651,2,2,68,216,34,34,34.000000,0.00000,108,108,108.000000,0.000000,1.387728e+03,19.545470,68217.000000,118148.381700,204643,4,4,4.00000,0.00000,4,4,4,4.00000,0.00000,4,4,0,0,0,0,40,64,9.772735,9.772735,34,108,63.600000,40.531469,1642.800000,0,0,0,0,0,0,0,0,1,79.500000,34.000000,108.000000,40,0,0,0,0,0,0,2,68,2,216,-1,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
152207,53,23394,1,1,57,73,57,57,57.000000,0.00000,73,73,73.000000,0.000000,5.556980e+03,85.492006,23394.000000,0.000000,23394,23394,0,0.00000,0.00000,0,0,0,0.00000,0.00000,0,0,0,0,0,0,32,20,42.746003,42.746003,57,73,62.333333,9.237604,85.333333,0,0,0,0,0,0,0,0,1,93.500000,57.000000,73.000000,32,0,0,0,0,0,0,1,57,1,73,-1,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
336531,53,190,2,2,60,264,30,30,30.000000,0.00000,132,132,132.000000,0.000000,1.705263e+06,21052.631580,63.333333,103.635580,183,3,4,4.00000,0.00000,4,4,3,3.00000,0.00000,3,3,0,0,0,0,64,64,10526.315790,10526.315790,30,132,70.800000,55.867701,3121.200000,0,0,0,0,0,0,0,0,1,88.500000,30.000000,132.000000,64,0,0,0,0,0,0,2,60,2,264,-1,-1,1,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [7]:
cols = list(df.columns)

for col in cols:
  if col[0] == ' ':
    df = df.rename(columns={col: col[1:]})

In [8]:
#df = df[imp_features]
cols = list(df.columns)

In [9]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df = df.reset_index(drop=True)

In [10]:
df.iloc[:,-1] = df.iloc[:, -1].apply(lambda x : 'Web' if x[:3] == 'Web' or x[:3] == 'Sql' else x)

Let's divide our dataframe into <i>features (X)</i> and <i>target (y)</i>.

In [11]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

We use Label Encoder to convert our String labels to integers to enable training the Regressor. (Machines only understand numbers)

In [12]:
le = LabelEncoder()
y = le.fit_transform(y)

We use <i>Standard Scaler</i> which scales the values of all features to a common specified range so that the models do not get skewed. (Otherwise big numbers automatically have greater impact <i>especially with neural networks</i>)

In [13]:
ss = StandardScaler()
X = ss.fit_transform(X)

In [14]:
X

array([[-4.16898415e-01, -4.27962240e-01,  1.15072316e-02, ...,
        -1.09516433e-01, -3.57071532e-01, -3.39184426e-01],
       [-4.38239842e-01, -4.39606497e-01, -9.82460718e-03, ...,
        -1.09516433e-01, -3.57071532e-01, -3.39184426e-01],
       [-4.38239842e-01, -4.33533934e-01, -9.82460718e-03, ...,
        -1.09516433e-01, -3.57071532e-01, -3.39184426e-01],
       ...,
       [-4.36762359e-01,  3.00120617e+00,  8.84075178e-03, ...,
        -1.01159224e-01,  6.13942856e-02,  9.17849818e-02],
       [-4.36762359e-01, -4.30722677e-01, -8.49136725e-03, ...,
        -1.09516433e-01, -3.57071532e-01, -3.39184426e-01],
       [-4.16898415e-01, -2.83694587e-01, -1.82516763e-03, ...,
        -1.09516433e-01, -3.57071532e-01, -3.39184426e-01]])

In [15]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

We split the data into the following:
- Training set 75%
- Test set 25%

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 21)

In [17]:
#savez_compressed('y_test.npz', y_test)

Let's create a dictionary which maps the integer labels to their corresponding string labels.

In [18]:
def get_integer_mapping(le):
    '''
    Return a dict mapping labels to their integer values
    from an SKlearn LabelEncoder
    le = a fitted SKlearn LabelEncoder
    '''
    res = {}
    for cl in le.classes_:
        res.update({cl:le.transform([cl])[0]})

    return res

integerMapping = get_integer_mapping(le)

Let's train the Random Forest Classifier.

In [19]:
rfClassifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
rfClassifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

Let's roughly get the accuracy of the model.

In [20]:
y_pred_rf = rfClassifier.predict(X_test)
accuracy_score(y_test, y_pred_rf)

0.9989292316919129

The Random Forest Classifier provides an accuracy of a whopping 99.89%.

In [21]:
savez_compressed('y_pred_rf.npz', y_pred_rf)

In [22]:
np.unique(y_pred_rf, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 array([567597,    350,  31944,   2556,  57792,   1385,   1361,   1962,
             4,     10,  39922,   1555,    531]))

We train the Naive Bayes Classifier.

In [23]:
nbClassifier = GaussianNB()
nbClassifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [24]:
y_pred_nb = nbClassifier.predict(X_test)
accuracy_score(y_test, y_pred_nb)

0.6987067325441426

We get an accuracy score of 69.87%.

In [25]:
savez_compressed('y_pred_nb.npz', y_pred_nb)

Let's build our neural network.

In [26]:
model_nn = Sequential()

model_nn.add(Dense(32, activation = 'relu', input_dim = 78))
model_nn.add(Dense(64, activation = 'relu'))

model_nn.add(Dense(13, activation = 'softmax'))

In [27]:
Adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [28]:
model_nn.compile(loss='categorical_crossentropy', optimizer=Adam, metrics=['accuracy'])

For softmax activation layer, we need one hot vectors and not integers.<br>
Hence, we create one hot vectors of the target.

In [29]:
y_train_nn = to_categorical(y_train)

We train our neural network.

In [30]:
model_nn.fit(X_train, y_train_nn, epochs = 2, validation_split = 0.2)

Epoch 1/2
53023/53023 [==============================] - 90s 2ms/step - loss: 0.1119 - accuracy: 0.9629 - val_loss: 0.0961 - val_accuracy: 0.9690
Epoch 2/2
53023/53023 [==============================] - 88s 2ms/step - loss: 0.1074 - accuracy: 0.9686 - val_loss: 0.0898 - val_accuracy: 0.9686


In [31]:
y_test_nn = to_categorical(y_test)
model_nn.evaluate(X_test, y_test_nn)

22093/22093 [==============================] - 23s 1ms/step - loss: 0.0929 - accuracy: 0.9689


[0.0929190069437027, 0.9689491391181946]

We get an accuracy score of 96.89%.

In [32]:
y_pred_nn = model_nn.predict(X_test)
y_pred_nn = np.argmax(y_pred_nn, axis=1)

In [33]:
savez_compressed('y_pred_nn.npz', y_pred_nn)

In [34]:
#model_nn.save("model_nn.h5")
#joblib.dump(rfClassifier, "RF_compressed.joblib", compress=3) 
#joblib.dump(nbClassifier, "NB_compressed.joblib", compress=3) 

Now that our models are trained, saved, and exported, we can move ahead with the evaluation.